# Probabilistic Bayesian Neural Networks

**Author:** [Khalid Salama](https://www.linkedin.com/in/khalid-salama-24403144/)<br>
**Date created:** 2021/01/15<br>
**Last modified:** 2021/01/15<br>
**Description:** Building probabilistic Bayesian neural network models with TensorFlow Probability.

## Introduction

Taking a probabilistic approach to deep learning allows to account for *uncertainty*,
so that models can assign less levels of confidence to incorrect predictions.
Sources of uncertainty can be found in the data, due to measurement error or
noise in the labels, or the model, due to insufficient data availability for
the model to learn effectively.


This example demonstrates how to build basic probabilistic Bayesian neural networks
to account for these two types of uncertainty.
We use [TensorFlow Probability](https://www.tensorflow.org/probability) library,
which is compatible with Keras API.

This example requires TensorFlow 2.3 or higher.
You can install Tensorflow Probability using the following command:

```python
pip install tensorflow-probability
```

## The dataset

We use the [Wine Quality](https://archive.ics.uci.edu/ml/datasets/wine+quality)
dataset, which is available in the [TensorFlow Datasets](https://www.tensorflow.org/datasets/catalog/wine_quality).
We use the red wine subset, which contains 4,898 examples.
The dataset has 11numerical physicochemical features of the wine, and the task
is to predict the wine quality, which is a score between 0 and 10.
In this example, we treat this as a regression task.

You can install TensorFlow Datasets using the following command:

```python
pip install tensorflow-datasets
```

## Setup

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_datasets as tfds
import tensorflow_probability as tfp

## Create training and evaluation datasets

Here, we load the `wine_quality` dataset using `tfds.load()`, and we convert
the target feature to float. Then, we shuffle the dataset and split it into
training and test sets. We take the first `train_size` examples as the train
split, and the rest as the test split.

In [2]:

def get_train_and_test_splits(train_size, batch_size=1):
    # We prefetch with a buffer the same size as the dataset because th dataset
    # is very small and fits into memory.
    dataset = (
        tfds.load(name="wine_quality", as_supervised=True, split="train")
        .map(lambda x, y: (x, tf.cast(y, tf.float32)))
        .prefetch(buffer_size=dataset_size)
        .cache()
    )
    # We shuffle with a buffer the same size as the dataset.
    train_dataset = (
        dataset.take(train_size).shuffle(buffer_size=train_size).batch(batch_size)
    )
    test_dataset = dataset.skip(train_size).batch(batch_size)

    return train_dataset, test_dataset


## Compile, train, and evaluate the model

In [3]:
hidden_units = [8, 8]
learning_rate = 0.001


def run_experiment(model, loss, train_dataset, test_dataset):

    model.compile(
        optimizer=keras.optimizers.RMSprop(learning_rate=learning_rate),
        loss=loss,
        metrics=[keras.metrics.RootMeanSquaredError()],
    )

    print("Start training the model...")
    model.fit(train_dataset, epochs=num_epochs, validation_data=test_dataset)
    print("Model training finished.")
    _, rmse = model.evaluate(train_dataset, verbose=0)
    print(f"Train RMSE: {round(rmse, 3)}")

    print("Evaluating model performance...")
    _, rmse = model.evaluate(test_dataset, verbose=0)
    print(f"Test RMSE: {round(rmse, 3)}")


## Create model inputs

In [4]:
FEATURE_NAMES = [
    "fixed acidity",
    "volatile acidity",
    "citric acid",
    "residual sugar",
    "chlorides",
    "free sulfur dioxide",
    "total sulfur dioxide",
    "density",
    "pH",
    "sulphates",
    "alcohol",
]


def create_model_inputs():
    inputs = {}
    for feature_name in FEATURE_NAMES:
        inputs[feature_name] = layers.Input(
            name=feature_name, shape=(1,), dtype=tf.float32
        )
    return inputs


## Experiment 1: standard neural network

We create a standard deterministic neural network model as a baseline.

In [6]:

def create_baseline_model():
    inputs = create_model_inputs()
    input_values = [value for _, value in sorted(inputs.items())]
    features = keras.layers.concatenate(input_values)
    features = layers.BatchNormalization()(features)

    # Create hidden layers with deterministic weights using the Dense layer.
    for units in hidden_units:
        features = layers.Dense(units, activation="sigmoid")(features)
    # The output is deterministic: a single point estimate.
    outputs = layers.Dense(units=1)(features)

    model = keras.Model(inputs=inputs, outputs=outputs)
    return model


Let's split the wine dataset into training and test sets, with 85% and 15% of
the examples, respectively.

In [7]:
dataset_size = 4898
batch_size = 256
train_size = int(dataset_size * 0.85)
train_dataset, test_dataset = get_train_and_test_splits(train_size, batch_size)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/4898 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/wine_quality/white/1.0.0.incompleteOTGG9X/wine_quality-train.tfrecord*...:…

Dataset wine_quality downloaded and prepared to /root/tensorflow_datasets/wine_quality/white/1.0.0. Subsequent calls will reuse this data.


Now let's train the baseline model. We use the `MeanSquaredError`
as the loss function.

In [8]:
num_epochs = 100
mse_loss = keras.losses.MeanSquaredError()
baseline_model = create_baseline_model()
run_experiment(baseline_model, mse_loss, train_dataset, test_dataset)

Start training the model...
Epoch 1/100
17/17 [==============================] - 7s 48ms/step - loss: 43.3616 - root_mean_squared_error: 6.5850 - val_loss: 42.8633 - val_root_mean_squared_error: 6.5470
Epoch 2/100
17/17 [==============================] - 0s 6ms/step - loss: 41.4578 - root_mean_squared_error: 6.4388 - val_loss: 41.2075 - val_root_mean_squared_error: 6.4193
Epoch 3/100
17/17 [==============================] - 0s 6ms/step - loss: 39.8801 - root_mean_squared_error: 6.3151 - val_loss: 39.5508 - val_root_mean_squared_error: 6.2889
Epoch 4/100
17/17 [==============================] - 0s 6ms/step - loss: 38.3810 - root_mean_squared_error: 6.1952 - val_loss: 37.9451 - val_root_mean_squared_error: 6.1600
Epoch 5/100
17/17 [==============================] - 0s 6ms/step - loss: 36.9361 - root_mean_squared_error: 6.0775 - val_loss: 36.4186 - val_root_mean_squared_error: 6.0348
Epoch 6/100
17/17 [==============================] - 0s 6ms/step - loss: 35.5373 - root_mean_squared_error

We take a sample from the test set use the model to obtain predictions for them.
Note that since the baseline model is deterministic, we get a single a
*point estimate* prediction for each test example, with no information about the
uncertainty of the model nor the prediction.

In [9]:
sample = 10
examples, targets = list(test_dataset.unbatch().shuffle(batch_size * 10).batch(sample))[
    0
]

predicted = baseline_model(examples).numpy()
for idx in range(sample):
    print(f"Predicted: {round(float(predicted[idx][0]), 1)} - Actual: {targets[idx]}")

Predicted: 5.7 - Actual: 5.0
Predicted: 5.9 - Actual: 7.0
Predicted: 5.9 - Actual: 6.0
Predicted: 5.8 - Actual: 5.0
Predicted: 6.0 - Actual: 7.0
Predicted: 5.7 - Actual: 5.0
Predicted: 5.5 - Actual: 5.0
Predicted: 5.9 - Actual: 5.0
Predicted: 6.3 - Actual: 6.0
Predicted: 6.2 - Actual: 7.0


## Experiment 2: Bayesian neural network (BNN)

The object of the Bayesian approach for modeling neural networks is to capture
the *epistemic uncertainty*, which is uncertainty about the model fitness,
due to limited training data.

The idea is that, instead of learning specific weight (and bias) *values* in the
neural network, the Bayesian approach learns weight *distributions*
- from which we can sample to produce an output for a given input -
to encode weight uncertainty.

Thus, we need to define prior and the posterior distributions of these weights,
and the training process is to learn the parameters of these distributions.

In [10]:
# Define the prior weight distribution as Normal of mean=0 and stddev=1.
# Note that, in this example, the weight prior distribution is not trainable,
# as we fix its parameters.
def prior(kernel_size, bias_size, dtype=None):
    n = kernel_size + bias_size
    prior_model = keras.Sequential(
        [
            tfp.layers.DistributionLambda(
                lambda t: tfp.distributions.MultivariateNormalDiag(
                    loc=tf.zeros(n), scale_diag=tf.ones(n)
                )
            )
        ]
    )
    return prior_model


# Define variational posterior weight distribution as multivariate Gaussian.
# Note that the learnable parameters for this distribution are the means,
# variances, and covariances.
def posterior(kernel_size, bias_size, dtype=None):
    n = kernel_size + bias_size
    posterior_model = keras.Sequential(
        [
            tfp.layers.VariableLayer(
                tfp.layers.MultivariateNormalTriL.params_size(n), dtype=dtype
            ),
            tfp.layers.MultivariateNormalTriL(n),
        ]
    )
    return posterior_model


We use the `tfp.layers.DenseVariational` layer instead of the standard
`keras.layers.Dense` layer in the neural network model.

In [11]:

def create_bnn_model(train_size):
    inputs = create_model_inputs()
    features = keras.layers.concatenate(list(inputs.values()))
    features = layers.BatchNormalization()(features)

    # Create hidden layers with weight uncertainty using the DenseVariational layer.
    for units in hidden_units:
        features = tfp.layers.DenseVariational(
            units=units,
            make_prior_fn=prior,
            make_posterior_fn=posterior,
            kl_weight=1 / train_size,
            activation="sigmoid",
        )(features)

    # The output is deterministic: a single point estimate.
    outputs = layers.Dense(units=1)(features)
    model = keras.Model(inputs=inputs, outputs=outputs)
    return model


The epistemic uncertainty can be reduced as we increase the size of the
training data. That is, the more data the BNN model sees, the more it is certain
about its estimates for the weights (distribution parameters).
Let's test this behaviour by training the BNN model on a small subset of
the training set, and then on the full training set, to compare the output variances.

### Train BNN  with a small training subset.

In [12]:
num_epochs = 500
train_sample_size = int(train_size * 0.3)
small_train_dataset = train_dataset.unbatch().take(train_sample_size).batch(batch_size)

bnn_model_small = create_bnn_model(train_sample_size)
run_experiment(bnn_model_small, mse_loss, small_train_dataset, test_dataset)

Start training the model...
Epoch 1/500
5/5 [==============================] - 4s 267ms/step - loss: 39.7277 - root_mean_squared_error: 6.3017 - val_loss: 36.5867 - val_root_mean_squared_error: 6.0471
Epoch 2/500
5/5 [==============================] - 0s 16ms/step - loss: 37.6421 - root_mean_squared_error: 6.1344 - val_loss: 40.8129 - val_root_mean_squared_error: 6.3872
Epoch 3/500
5/5 [==============================] - 0s 15ms/step - loss: 36.6266 - root_mean_squared_error: 6.0508 - val_loss: 33.9256 - val_root_mean_squared_error: 5.8233
Epoch 4/500
5/5 [==============================] - 0s 13ms/step - loss: 37.8790 - root_mean_squared_error: 6.1533 - val_loss: 37.7857 - val_root_mean_squared_error: 6.1460
Epoch 5/500
5/5 [==============================] - 0s 12ms/step - loss: 36.7867 - root_mean_squared_error: 6.0640 - val_loss: 33.6143 - val_root_mean_squared_error: 5.7968
Epoch 6/500
5/5 [==============================] - 0s 13ms/step - loss: 36.4541 - root_mean_squared_error: 6.03

Since we have trained a BNN model, the model produces a different output each time
we call it with the same input, since each time a new set of weights are sampled
from the distributions to construct the network and produce an output.
The less certain the mode weights are, the more variability (wider range) we will
see in the outputs of the same inputs.

In [13]:

def compute_predictions(model, iterations=100):
    predicted = []
    for _ in range(iterations):
        predicted.append(model(examples).numpy())
    predicted = np.concatenate(predicted, axis=1)

    prediction_mean = np.mean(predicted, axis=1).tolist()
    prediction_min = np.min(predicted, axis=1).tolist()
    prediction_max = np.max(predicted, axis=1).tolist()
    prediction_range = (np.max(predicted, axis=1) - np.min(predicted, axis=1)).tolist()

    for idx in range(sample):
        print(
            f"Predictions mean: {round(prediction_mean[idx], 2)}, "
            f"min: {round(prediction_min[idx], 2)}, "
            f"max: {round(prediction_max[idx], 2)}, "
            f"range: {round(prediction_range[idx], 2)} - "
            f"Actual: {targets[idx]}"
        )


compute_predictions(bnn_model_small)

Predictions mean: 5.76, min: 5.07, max: 6.28, range: 1.21 - Actual: 5.0
Predictions mean: 6.19, min: 5.57, max: 6.41, range: 0.83 - Actual: 7.0
Predictions mean: 6.24, min: 5.87, max: 6.42, range: 0.55 - Actual: 6.0
Predictions mean: 5.48, min: 4.97, max: 6.06, range: 1.08 - Actual: 5.0
Predictions mean: 6.16, min: 5.64, max: 6.4, range: 0.76 - Actual: 7.0
Predictions mean: 5.85, min: 5.28, max: 6.27, range: 0.99 - Actual: 5.0
Predictions mean: 5.28, min: 4.57, max: 6.07, range: 1.5 - Actual: 5.0
Predictions mean: 6.06, min: 5.68, max: 6.32, range: 0.64 - Actual: 5.0
Predictions mean: 6.31, min: 5.85, max: 6.45, range: 0.6 - Actual: 6.0
Predictions mean: 6.21, min: 5.77, max: 6.38, range: 0.61 - Actual: 7.0


### Train BNN  with the whole training set.

In [14]:
num_epochs = 500
bnn_model_full = create_bnn_model(train_size)
run_experiment(bnn_model_full, mse_loss, train_dataset, test_dataset)

compute_predictions(bnn_model_full)

Start training the model...
Epoch 1/500
17/17 [==============================] - 3s 45ms/step - loss: 42.7095 - root_mean_squared_error: 6.5349 - val_loss: 40.9651 - val_root_mean_squared_error: 6.4000
Epoch 2/500
17/17 [==============================] - 0s 7ms/step - loss: 39.8885 - root_mean_squared_error: 6.3154 - val_loss: 37.7047 - val_root_mean_squared_error: 6.1400
Epoch 3/500
17/17 [==============================] - 0s 7ms/step - loss: 39.6687 - root_mean_squared_error: 6.2980 - val_loss: 41.9938 - val_root_mean_squared_error: 6.4799
Epoch 4/500
17/17 [==============================] - 0s 8ms/step - loss: 38.5245 - root_mean_squared_error: 6.2065 - val_loss: 33.3890 - val_root_mean_squared_error: 5.7779
Epoch 5/500
17/17 [==============================] - 0s 8ms/step - loss: 36.0118 - root_mean_squared_error: 6.0006 - val_loss: 33.3950 - val_root_mean_squared_error: 5.7784
Epoch 6/500
17/17 [==============================] - 0s 7ms/step - loss: 35.4188 - root_mean_squared_error

Notice that the model trained with the full training dataset shows smaller range
(uncertainty) in the prediction values for the same inputs, compared to the model
trained with a subset of the training dataset.

## Experiment 3: probabilistic Bayesian neural network

So far, the output of the standard and the Bayesian NN models that we built is
deterministic, that is, produces a point estimate as a prediction for a given example.
We can create a probabilistic NN by letting the model output a distribution.
In this case, the model captures the *aleatoric uncertainty* as well,
which is due to irreducible noise in the data, or to the stochastic nature of the
process generating the data.

In this example, we model the output as a `IndependentNormal` distribution,
with learnable mean and variance parameters. If the task was classification,
we would have used `IndependentBernoulli` with binary classes, and `OneHotCategorical`
with multiple classes, to model distribution of the model output.

In [15]:
def create_probablistic_bnn_model(train_size):
    inputs = create_model_inputs()
    features = keras.layers.concatenate(list(inputs.values()))
    features = layers.BatchNormalization()(features)

    # Create hidden layers with weight uncertainty using the DenseVariational layer.
    for units in hidden_units:
        features = tfp.layers.DenseVariational(
            units=units,
            make_prior_fn=prior,
            make_posterior_fn=posterior,
            kl_weight=1 / train_size,
            activation="sigmoid",
        )(features)

    # Create a probabilistic output (Normal distribution), and use the `Dense` layer
    # to produce the parameters of the distribution.
    # We set units=2 to learn both the mean and the variance of the Normal distribution.
    distribution_params = layers.Dense(units=2)(features)
    outputs = tfp.layers.IndependentNormal(1)(distribution_params)

    model = keras.Model(inputs=inputs, outputs=outputs)
    return model


Since the output of the model is a distribution, rather than a point estimate,
we use the [negative loglikelihood](https://en.wikipedia.org/wiki/Likelihood_function)
as our loss function to compute how likely to see the true data (targets) from the
estimated distribution produced by the model.

In [16]:

def negative_loglikelihood(targets, estimated_distribution):
    return -estimated_distribution.log_prob(targets)


num_epochs = 1000
prob_bnn_model = create_probablistic_bnn_model(train_size)
run_experiment(prob_bnn_model, negative_loglikelihood, train_dataset, test_dataset)

Start training the model...
Epoch 1/1000
17/17 [==============================] - 3s 49ms/step - loss: 9.2229 - root_mean_squared_error: 5.2780 - val_loss: 7.8589 - val_root_mean_squared_error: 5.3663
Epoch 2/1000
17/17 [==============================] - 0s 7ms/step - loss: 10.2221 - root_mean_squared_error: 5.3410 - val_loss: 5.1658 - val_root_mean_squared_error: 4.9552
Epoch 3/1000
17/17 [==============================] - 0s 7ms/step - loss: 7.3521 - root_mean_squared_error: 5.1729 - val_loss: 5.7378 - val_root_mean_squared_error: 4.9185
Epoch 4/1000
17/17 [==============================] - 0s 7ms/step - loss: 6.2595 - root_mean_squared_error: 4.9380 - val_loss: 9.4447 - val_root_mean_squared_error: 4.9931
Epoch 5/1000
17/17 [==============================] - 0s 7ms/step - loss: 5.1490 - root_mean_squared_error: 4.9832 - val_loss: 5.0394 - val_root_mean_squared_error: 4.8866
Epoch 6/1000
17/17 [==============================] - 0s 7ms/step - loss: 5.3542 - root_mean_squared_error: 4.

Now let's produce an output from the model given the test examples.

---


The output is now a distribution, and we can use its mean and variance
to compute the confidence intervals (CI) of the prediction.

In [17]:
prediction_distribution = prob_bnn_model(examples)
prediction_mean = prediction_distribution.mean().numpy().tolist()
prediction_stdv = prediction_distribution.stddev().numpy()

# The 95% CI is computed as mean ± (1.96 * stdv)
upper = (prediction_mean + (1.96 * prediction_stdv)).tolist()
lower = (prediction_mean - (1.96 * prediction_stdv)).tolist()
prediction_stdv = prediction_stdv.tolist()

for idx in range(sample):
    print(
        f"Prediction mean: {round(prediction_mean[idx][0], 2)}, "
        f"stddev: {round(prediction_stdv[idx][0], 2)}, "
        f"95% CI: [{round(upper[idx][0], 2)} - {round(lower[idx][0], 2)}]"
        f" - Actual: {targets[idx]}"
    )

Prediction mean: 5.8, stddev: 0.7, 95% CI: [7.18 - 4.43] - Actual: 5.0
Prediction mean: 6.53, stddev: 0.86, 95% CI: [8.21 - 4.85] - Actual: 7.0
Prediction mean: 6.2, stddev: 0.76, 95% CI: [7.7 - 4.71] - Actual: 6.0
Prediction mean: 5.07, stddev: 0.65, 95% CI: [6.34 - 3.8] - Actual: 5.0
Prediction mean: 6.47, stddev: 0.85, 95% CI: [8.14 - 4.81] - Actual: 7.0
Prediction mean: 6.06, stddev: 0.74, 95% CI: [7.51 - 4.6] - Actual: 5.0
Prediction mean: 5.19, stddev: 0.66, 95% CI: [6.47 - 3.9] - Actual: 5.0
Prediction mean: 5.73, stddev: 0.71, 95% CI: [7.13 - 4.33] - Actual: 5.0
Prediction mean: 6.61, stddev: 0.87, 95% CI: [8.31 - 4.91] - Actual: 6.0
Prediction mean: 6.21, stddev: 0.78, 95% CI: [7.73 - 4.68] - Actual: 7.0
